## NENEURAL NETWORK FOR BAKERY SALES PREDICTION

In [70]:
#Data handling
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import holidays
#Date handling
from datetime import datetime
    
#processing
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
    
#Deep learning
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Activation, Flatten,Concatenate
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
    
#Metrics
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_percentage_error



In [3]:
train_df = pd.read_csv('../0_DataPreparation/Project_data/train.csv')
test_df = pd.read_csv('../0_DataPreparation/Project_data/test.csv')
weather_df = pd.read_csv('../0_DataPreparation/Project_data/wetter.csv')
kiwo_df = pd.read_csv('../0_DataPreparation/Project_data/kiwo.csv')

In [4]:
#Passing the date column to datetime
train_df['Datum'] = pd.to_datetime(train_df['Datum'], errors='coerce')
test_df['Datum'] = pd.to_datetime(test_df['Datum'], errors='coerce')
weather_df['Datum'] = pd.to_datetime(weather_df['Datum'], errors='coerce')
kiwo_df['Datum'] = pd.to_datetime(kiwo_df['Datum'], errors= 'coerce')

In [5]:
print(train_df['Datum'].dtype)
print(test_df['Datum'].dtype)
print(weather_df['Datum'].dtype)
print(kiwo_df['Datum'].dtype)

datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]


## Preprocessing the Data

In [6]:
print("Train missing values:", train_df.isnull().sum().sum())
print("Test missing values:", test_df.isnull().sum().sum())
print("Weather missing values:", weather_df.isnull().sum().sum())
print("Kiwo missing values:", kiwo_df.isnull().sum().sum())

Train missing values: 0
Test missing values: 0
Weather missing values: 679
Kiwo missing values: 0


In [7]:
print('Datum' in weather_df.columns)

True


In [8]:
print("Index name:", weather_df.index.name)


Index name: None


In [9]:
print(train_df['Datum'].isna().sum())
print(test_df['Datum'].isna().sum())
print(weather_df['Datum'].isna().sum())
print(kiwo_df['Datum'].isna().sum())


0
0
0
0


In [10]:
# parse numeric columns correctly (in case of text/corruption)
weather_df['Bewoelkung'] = pd.to_numeric(weather_df['Bewoelkung'], errors='coerce')
weather_df['Temperatur'] = pd.to_numeric(weather_df['Temperatur'], errors='coerce')
weather_df['Windgeschwindigkeit'] = pd.to_numeric(weather_df['Windgeschwindigkeit'], errors='coerce')
weather_df['Wettercode'] = pd.to_numeric(weather_df['Wettercode'], errors='coerce')

In [11]:
weather_df.set_index('Datum', inplace=True)
full_dates = pd.date_range(start=weather_df.index.min(), end=weather_df.index.max(), freq='D')
weather_df = weather_df.reindex(full_dates)
weather_df = weather_df.infer_objects(copy=False)
weather_df.interpolate(method='linear', inplace=True)
weather_df.bfill(inplace=True)
weather_df.ffill(inplace=True)
weather_df.reset_index(names='Datum', inplace=True)


In [12]:
print(weather_df.isnull().sum())


Datum                  0
Bewoelkung             0
Temperatur             0
Windgeschwindigkeit    0
Wettercode             0
dtype: int64


In [24]:
# Merge weather and kiwo data into train and test sets
train_df = pd.merge(train_df, weather_df, on='Datum', how='left')
train_df = pd.merge(train_df, kiwo_df, on='Datum', how='left')
test_df  = pd.merge(test_df, weather_df, on='Datum', how='left')
test_df  = pd.merge(test_df, kiwo_df, on='Datum', how='left')

In [25]:
# Fill missing festival flags with 0 (non-festival days)
train_df['KielerWoche'] = train_df['KielerWoche'].fillna(0)
test_df['KielerWoche']  = test_df['KielerWoche'].fillna(0)


In [26]:
print("Train missing values after merge:", train.isnull().sum().sum())
print("Test missing values after merge:", test.isnull().sum().sum())
print(train.head())
print(test.head())

Train missing values after merge: 0
Test missing values after merge: 0
        id      Datum  Warengruppe      Umsatz  Bewoelkung  Temperatur  \
0  1307011 2013-07-01            1  148.828353         6.0     17.8375   
1  1307021 2013-07-02            1  159.793757         3.0     17.3125   
2  1307031 2013-07-03            1  111.885594         7.0     21.0750   
3  1307041 2013-07-04            1  168.864941         7.0     18.8500   
4  1307051 2013-07-05            1  171.280754         5.0     19.9750   

   Windgeschwindigkeit  Wettercode  KielerWoche  Year  Month  DayOfWeek  \
0                 15.0       20.00          0.0  2013      7          0   
1                 10.0       40.50          0.0  2013      7          1   
2                  6.0       61.00          0.0  2013      7          2   
3                  7.0       20.00          0.0  2013      7          3   
4                 12.0       16.25          0.0  2013      7          4   

   is_weekend  
0           0  
1

## FEATURE ENGINEERING

In [27]:
#Extracting date features
train_df['Year'] = train_df['Datum'].dt.year
train_df['Month'] = train_df['Datum'].dt.month
train_df['DayOfWeek'] = train_df['Datum'].dt.dayofweek   #0 = Monday, 6 = Sunday
train_df['is_weekend'] = train_df['DayOfWeek'].isin([5, 6]).astype(int)

test_df['Year'] = test_df['Datum'].dt.year
test_df['Month'] = test_df['Datum'].dt.month
test_df['DayOfWeek'] = test_df['Datum'].dt.dayofweek
test_df['is_weekend'] = test_df['DayOfWeek'].isin([5, 6]).astype(int)



In [28]:
train_df = pd.get_dummies(train_df, columns=['DayOfWeek', 'Month'], drop_first=True)
test_df = pd.get_dummies(test_df, columns=['DayOfWeek', 'Month'], drop_first=True)


In [29]:
print(train_df.columns)
print(test_df.columns)

Index(['id', 'Datum', 'Warengruppe', 'Umsatz', 'Bewoelkung', 'Temperatur',
       'Windgeschwindigkeit', 'Wettercode', 'KielerWoche', 'Year',
       'is_weekend', 'DayOfWeek_1', 'DayOfWeek_2', 'DayOfWeek_3',
       'DayOfWeek_4', 'DayOfWeek_5', 'DayOfWeek_6', 'Month_2', 'Month_3',
       'Month_4', 'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9',
       'Month_10', 'Month_11', 'Month_12'],
      dtype='object')
Index(['id', 'Datum', 'Warengruppe', 'Bewoelkung', 'Temperatur',
       'Windgeschwindigkeit', 'Wettercode', 'KielerWoche', 'Year',
       'is_weekend', 'DayOfWeek_1', 'DayOfWeek_2', 'DayOfWeek_3',
       'DayOfWeek_4', 'DayOfWeek_5', 'DayOfWeek_6', 'Month_2', 'Month_3',
       'Month_4', 'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9',
       'Month_10', 'Month_11', 'Month_12'],
      dtype='object')


In [31]:
print(train_df.head(2))

        id      Datum  Warengruppe      Umsatz  Bewoelkung  Temperatur  \
0  1307011 2013-07-01            1  148.828353         6.0     17.8375   
1  1307021 2013-07-02            1  159.793757         3.0     17.3125   

   Windgeschwindigkeit  Wettercode  KielerWoche  Year  ...  Month_3  Month_4  \
0                 15.0        20.0          0.0  2013  ...    False    False   
1                 10.0        40.5          0.0  2013  ...    False    False   

   Month_5  Month_6  Month_7  Month_8  Month_9  Month_10  Month_11  Month_12  
0    False    False     True    False    False     False     False     False  
1    False    False     True    False    False     False     False     False  

[2 rows x 28 columns]


In [46]:
# Add missing dummy columns to test
for col in train_df.columns:
    if col.startswith('dayofweek_') or col.startswith('month_'):
        if col not in test_df.columns:
            test_df[col] = 0

# Drop any extras in test not in train
for col in test_df.columns:
    if (col.startswith('dayofweek_') or col.startswith('month_')) and col not in train_df.columns:
        train_df[col] = 0


In [47]:
train_df.drop(columns='Datum', inplace=True, errors='ignore')
test_df.drop(columns='Datum', inplace=True, errors='ignore')


In [ ]:
# Encode categorical features

#encoder = LabelEncoder()
#all_groups = pd.concat([train_df['Warengruppe'], test_df['Warengruppe']])

#encoder.fit(all_groups)

# Encode Warengruppe 
if 'Warengruppe_code' not in train_df.columns:
    if 'Warengruppe' in train_df.columns:
        train_df['Warengruppe_code'] = train_df['Warengruppe'] - 1
        train_df.drop(columns='Warengruppe', inplace=True, errors='ignore')

if 'Warengruppe_code' not in test_df.columns:
    if 'Warengruppe' in test_df.columns:
        test_df['Warengruppe_code'] = test_df['Warengruppe'] - 1
        test_df.drop(columns='Warengruppe', inplace=True, errors='ignore')


In [53]:
print(train_df.columns)


Index(['id', 'Umsatz', 'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit',
       'Wettercode', 'KielerWoche', 'Year', 'is_weekend', 'DayOfWeek_1',
       'DayOfWeek_2', 'DayOfWeek_3', 'DayOfWeek_4', 'DayOfWeek_5',
       'DayOfWeek_6', 'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6',
       'Month_7', 'Month_8', 'Month_9', 'Month_10', 'Month_11', 'Month_12',
       'Warengruppe_code'],
      dtype='object')


In [56]:
#  German holidays (Schleswig-Holstein covers Kiel)
german_holidays = holidays.Germany(prov='SH', years=range(2012, 2020))

# Set of all holiday dates
holiday_dates = set(german_holidays.keys())

In [60]:
if 'Datum' not in train_df.columns and 'Datum' in train.columns:
    train_df['Datum'] = train['Datum']
if 'Datum' not in test_df.columns and 'Datum' in test.columns:
    test_df['Datum'] = test['Datum']

train_df['is_holiday'] = train_df['Datum'].isin(holiday_dates).astype(int)
test_df['is_holiday'] = test_df['Datum'].isin(holiday_dates).astype(int)

train_df.drop(columns='Datum', inplace=True, errors='ignore')
test_df.drop(columns='Datum', inplace=True, errors='ignore')


/var/folders/0v/wz051f795j13ts216fq3zpwc0000gn/T/ipykernel_1113/2630270668.py:6: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  train_df['is_holiday'] = train_df['Datum'].isin(holiday_dates).astype(int)
/var/folders/0v/wz051f795j13ts216fq3zpwc0000gn/T/ipykernel_1113/2630270668.py:7: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  test_df['is_holiday'] = test_df['Datum'].isin(holiday_dates).astype(int)


In [61]:
train_df['is_off'] = ((train_df['is_weekend'] == 1) | (train_df['is_holiday'] == 1)).astype(int)
test_df['is_off'] = ((test_df['is_weekend'] == 1) | (test_df['is_holiday'] == 1)).astype(int)


## Split Data into  Training and Validation Sets

In [64]:
# Restore datum column if it was dropped
if 'Datum' not in train_df.columns and 'Datum' in train.columns:
    train_df['Datum'] = train['Datum']

# Now convert to datetime
train_df['Datum'] = pd.to_datetime(train_df['Datum'], errors='coerce')


# Define date ranges
training_start_date = '2013-07-01'
training_end_date = '2017-07-31'

validation_start_date = '2017-08-01'
validation_end_date = '2018-07-31'

# Create masks
train_mask = (train_df['Datum'] >= training_start_date) & (train_df['Datum'] <= training_end_date)
val_mask = (train_df['Datum'] >= validation_start_date) & (train_df['Datum'] <= validation_end_date)

# Apply masks
train_split = train_df[train_mask].copy()
val_split = train_df[val_mask].copy()


In [65]:
#Seperating features and target variable
X_train = train_split.drop(columns='Umsatz')
y_train = train_split['Umsatz']

X_val = val_split.drop(columns='Umsatz')
y_val = val_split['Umsatz']


In [67]:
#Scaling the features

# Drop the categorical column before scaling
num_features = X_train.drop(columns=['Warengruppe_code']).select_dtypes(include='number').columns

# Initialize scaler
scaler = StandardScaler()

# Scale numeric features
X_train_scaled = X_train.copy()
X_val_scaled = X_val.copy()

X_train_scaled[num_features] = scaler.fit_transform(X_train[num_features])
X_val_scaled[num_features] = scaler.transform(X_val[num_features])


In [82]:
# Prepare categorical input for embedding
X_train_cat = X_train['Warengruppe_code'].astype('int32').values
X_val_cat   = X_val['Warengruppe_code'].astype('int32').values

# Remove datetime and embedding columns from numeric features
X_train_num_df = X_train.drop(columns=['Warengruppe_code'], errors='ignore')
X_val_num_df   = X_val.drop(columns=['Warengruppe_code'], errors='ignore')

# Exclude datetime columns before casting
X_train_num_df = X_train_num_df.select_dtypes(exclude=['datetime', 'datetime64[ns]'])
X_val_num_df   = X_val_num_df.select_dtypes(exclude=['datetime', 'datetime64[ns]'])

# Convert to float32 numpy arrays
X_train_num = X_train_num_df.astype('float32').values
X_val_num   = X_val_num_df.astype('float32').values


## NEURAL NETWORK

In [84]:


# Defining Input Layers

numeric_input = layers.Input(shape=(X_train_num.shape[1],), name="numeric_input")
cat_input     = layers.Input(shape=(1,), name="warengruppe_input")


# Embedding for Categorical Input
embed_out = layers.Embedding(input_dim=6, output_dim=3, name="warengruppe_embed")(cat_input)
embed_flat = layers.Flatten()(embed_out)

# Concatenate Inputs
concat = layers.concatenate([numeric_input, embed_flat], name="concat_layer")

# Hidden Layers
dense1 = layers.Dense(64, activation='relu', name="dense_1")(concat)
dense1 = layers.Dropout(0.3)(dense1)
dense2 = layers.Dense(32, activation='relu', name="dense_2")(dense1)
dense2 = layers.Dropout(0.2)(dense2)

# Output Layer


output = layers.Dense(1, activation='linear', name="output")(dense2)



In [85]:
# Build and Compile Model
model = Model(inputs=[numeric_input, cat_input], outputs=output)
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=[tf.keras.metrics.MeanAbsolutePercentageError()]
)
model.summary()

# Early Stopping Callback

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ warengruppe_input   │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ warengruppe_embed   │ (None, 1, 3)      │         18 │ warengruppe_inpu… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ numeric_input       │ (None, 27)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 3)         │          0 │ warengruppe_embe… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concat_layer        │ (None, 30)        │          0 │ numeric_input[0]… │
│ (Concatenate)       │                   │            │ flatten_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      1,984 │ concat_layer[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 64)        │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │      2,080 │ dropout_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 32)        │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │         33 │ dropout_7[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,115 (16.07 KB)

 Trainable params: 4,115 (16.07 KB)

 Non-trainable params: 0 (0.00 B)

In [86]:
print(X_train.drop(columns='Warengruppe_code').shape)  


(7493, 28)


In [87]:

# Prepare numeric inputs (drop embedding + datetime cols)
X_train_num_df = X_train.drop(columns=['Warengruppe_code'], errors='ignore')
X_val_num_df   = X_val.drop(columns=['Warengruppe_code'], errors='ignore')

# Remove any datetime columns
X_train_num_df = X_train_num_df.select_dtypes(exclude=['datetime', 'datetime64[ns]'])
X_val_num_df   = X_val_num_df.select_dtypes(exclude=['datetime', 'datetime64[ns]'])

#  Convert to float32 numpy arrays
X_train_num = X_train_num_df.astype('float32').values
X_val_num   = X_val_num_df.astype('float32').values

#  Convert categorical input for embedding to int32
X_train_cat = np.array(X_train['Warengruppe_code']).astype('int32')
X_val_cat   = np.array(X_val['Warengruppe_code']).astype('int32')

#  Convert target to float32
y_train = np.array(y_train).astype('float32')
y_val   = np.array(y_val).astype('float32')


In [88]:
# Train the Model
history = model.fit(
    [X_train_num, X_train_cat],
    y_train,
    validation_data=([X_val_num, X_val_cat], y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)




Epoch 1/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 11711396864.0000 - mean_absolute_percentage_error: 60352.5391 - val_loss: 5094504.5000 - val_mean_absolute_percentage_error: 1772.5833
Epoch 2/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 636953472.0000 - mean_absolute_percentage_error: 13953.2852 - val_loss: 55420.0078 - val_mean_absolute_percentage_error: 100.0055
Epoch 3/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 153192624.0000 - mean_absolute_percentage_error: 6139.0244 - val_loss: 55416.6992 - val_mean_absolute_percentage_error: 99.9991
Epoch 4/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 63397292.0000 - mean_absolute_percentage_error: 3172.6948 - val_loss: 55413.3047 - val_mean_absolute_percentage_error: 99.9925
Epoch 5/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 33618876.0000 - mean_absolute_percentage_error: 2051.8027 - val_loss: 55410.0859 - val_mean_absolute_percentage_error: 99.9862
Epoch 6/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step -

In [89]:
# Evaluate Performance

val_loss, val_mape = model.evaluate([X_val_num, X_val_cat], y_val)
print(f"\n✅ Validation MAPE: {val_mape:.4f}")


58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 16647.5898 - mean_absolute_percentage_error: 61.5091

✅ Validation MAPE: 70.3247


In [98]:
import matplotlib.pyplot as plt
import pandas as pd

# Reconstruct feature names
feature_names = [f"num_{i}" for i in range(X_val_num.shape[1])] + ["Warengruppe_code"]

# Append both importances (numeric + categorical)
importances = np.append(result.importances_mean[:X_val_num.shape[1]], result.importances_mean[-1])

# Plot
pd.Series(importances, index=feature_names).sort_values().plot(kind='barh')
plt.title("Permutation Feature Importance")
plt.tight_layout()
plt.show()


NameError: name 'result' is not defined